In [8]:
import os,h5py
import numpy as np

with open('/capstor/store/cscs/userlab/s1174/lyan/code/glwc2/project2/02_discNJN_1D/dataPrepare/cB211.072.64/data_aux/cfgs_N=200,85,64_jlsc') as f:
    cfgs_Nj=f.read().splitlines()

path='/capstor/store/cscs/userlab/s1174/lyan/code/scratch/run/Nsgm/cB211.072.64/jPP/runAux/cfgs_run1'
with open(path) as f:
    cfgs_jPP=f.read().splitlines()
    
path='/capstor/store/cscs/userlab/s1174/lyan/code/scratch/run/Nsgm/cB211.072.64/loop/runAux/cfgs_run1'
with open(path) as f:
    cfgs_loop=f.read().splitlines()
    
path='/capstor/store/cscs/userlab/s1174/fpittler/runs/njnsigma_physical_point_64/'
pathb='/capstor/store/cscs/userlab/s1174/fpittler/runs/njnsigma_physical_point_64_b/'
cfgs_njnsgm=['a'+cfg for cfg in os.listdir(path)]+['b'+cfg for cfg in os.listdir(pathb)]

# path='/capstor/store/cscs/userlab/s1174/fpittler/runs/njn_physical_point_64/'
# pathb='/capstor/store/cscs/userlab/s1174/fpittler/runs/njn_physical_point_64_b/'
# cfgs_njn=['a'+cfg for cfg in os.listdir(path)]+['b'+cfg for cfg in os.listdir(pathb)]

path='/capstor/store/cscs/userlab/s1174/fpittler/runs/njn_physical_point_64/'
pathb='/capstor/scratch/cscs/fpittler/run/run_64_NJN_b/'
cfgs_njn=['a'+cfg for cfg in os.listdir(path)]+['b'+file[7:11] for file in os.listdir(pathb) if 'dt20_protondn.h5' in file]

cfgs=(set(cfgs_Nj)-set(cfgs_jPP)-set(cfgs_loop)).intersection(cfgs_njnsgm).intersection(cfgs_njn)
cfgs=list(cfgs)
cfgs.sort()

print(len(cfgs))

with open('data_aux/cfgs_run_todo','w') as f:
    f.write('\n'.join(cfgs))

640


In [59]:
import os,h5py
import numpy as np

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]


path='/p/project/ngff/li47/code/projectData2/01_Nsgm/cB211.072.64_base/data_pre_NJN'
cfgs=[cfg2new(cfg) for cfg in os.listdir(path)]
cfgs.sort()
with open('data_aux/cfgs_NJN_pre','w') as f:
    f.write('\n'.join(cfgs))

In [80]:
import os,h5py
import numpy as np

path='/p/project/ngff/li47/code/glwc2/project2/01_Nsgm/dataPrepare/cB211.072.64_base/data_aux/cfgs_NJN_pre'
with open(path,'r') as f:
    cfgs=f.read().splitlines()

dat={}    

path='/p/project/ngff/li47/code/projectData2/01_Nsgm/cB211.072.64_base/data_out_NJN/'
for i,cfg in enumerate(cfgs):
    print(i,len(cfgs),end='               \r')
    with h5py.File(f'{path}{cfg}/NJN.h5') as f:
        moms=[list(mom) for mom in f['moms']]
        ind=moms.index([0,0,0])
        for dt in f.keys():
            if not dt.startswith('dt'):
                continue
            for fla in f[dt].keys():
                if (dt,fla) not in dat.keys():
                    dat[(dt,fla)]=[]
                for src in f[f'{dt}/{fla}/Local/P4'].keys():
                    t=f[f'{dt}/{fla}/Local/P4/{src}'][:]
                    t=t[:,ind]
                    dat[(dt,fla)].append(t)
    # break
for ky in dat.keys():
    dat[ky]=np.mean(dat[ky],axis=0)

In [90]:
import os,h5py
import numpy as np


path='/p/project/ngff/li47/code/projectData/01_Nsgm/thrp-conn.h5'

with h5py.File(path) as f:
    for tf in [12,14,16,18,20][:]:
        print(tf)
        t=np.mean([f[f'gS/up/dt{tf}/{cfg}'][:]+f[f'gS/dn/dt{tf}/{cfg}'][:] for cfg in f[f'gS/up/dt{tf}'].keys()],axis=0)
        
        t2=dat[(f'dt{tf}','up')]+dat[(f'dt{tf}','dn')]
        t2=np.real(t2[:,0])
        print(t)
        print(t2/t)

12
[1.35123388e-09 1.96450816e-09 2.19924784e-09 2.35646953e-09
 2.45794896e-09 2.51443327e-09 2.53983409e-09 2.52150993e-09
 2.44944658e-09 2.33221265e-09 2.16769940e-09 1.92965393e-09
 1.32306587e-09]
[0.74678933 0.82457202 0.84725126 0.86039802 0.86809337 0.87565501
 0.87867741 0.88012754 0.88327236 0.88393153 0.88611689 0.88789233
 0.86740662]
14
[6.94708932e-10 9.87570252e-10 1.10467439e-09 1.18298305e-09
 1.23988023e-09 1.28034811e-09 1.30666300e-09 1.31737348e-09
 1.30824130e-09 1.28339487e-09 1.24299099e-09 1.18612095e-09
 1.10449709e-09 9.87803620e-10 6.92964113e-10]
[1.01407174 1.01144094 1.01113752 1.01112341 1.01089787 1.01205118
 1.01113806 1.00632046 1.00301096 1.00313567 1.00694814 1.01047409
 1.00961325 1.00626883 0.99695356]
16
[3.24172620e-10 4.58719696e-10 5.13125815e-10 5.50645426e-10
 5.78323274e-10 5.99941760e-10 6.15706437e-10 6.26438925e-10
 6.28999513e-10 6.25556921e-10 6.16293960e-10 6.01146169e-10
 5.78296472e-10 5.45868875e-10 5.02831293e-10 4.44501287e-10
 